In [1]:
import numpy as np
import pandas as pd
import keras
import keras.backend as K
from keras.optimizers import Adam
from keras.models import Sequential
from keras.utils import Sequence
from keras.layers import *
import matplotlib.pyplot as plt
from google.colab import files
import io 
from google.colab import drive

In [2]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [3]:
path = "/content/drive/MyDrive/Projects 2021/Sudoku ML Solver/sudoku.csv"
df = pd.read_csv(path)

In [4]:
df_sudoku = pd.DataFrame({"quizzes":df["puzzle"],"solutions":df["solution"]})
df_sudoku.head(5)

,quizzes,solutions
0,0700000430400096108006349000940520003584600200...,6795182435437296188216349577943521863584617292...
1,3010865040465210705000000014008000020803479000...,3719865248465213795924738614638197522853479167...
2,0483015603600080909106700030200009355090102006...,7483915623652487919126754834217869355894132766...
3,0083170000042051090000400703271609049014500000...,2983176457642851391539462783271689549814537266...
4,0408906300001368208007405190004670524500207002...,1428956379751368248367425193984671524513287962...


In [5]:
print("Quiz:\n",np.array(list(map(int,list(df_sudoku['quizzes'][0])))).reshape(9,9))
print("Solution:\n",np.array(list(map(int,list(df_sudoku['solutions'][0])))).reshape(9,9))

Quiz:
 [[0 7 0 0 0 0 0 4 3]
 [0 4 0 0 0 9 6 1 0]
 [8 0 0 6 3 4 9 0 0]
 [0 9 4 0 5 2 0 0 0]
 [3 5 8 4 6 0 0 2 0]
 [0 0 0 8 0 0 5 3 0]
 [0 8 0 0 7 0 0 9 1]
 [9 0 2 1 0 0 0 0 5]
 [0 0 7 0 4 0 8 0 2]]
Solution:
 [[6 7 9 5 1 8 2 4 3]
 [5 4 3 7 2 9 6 1 8]
 [8 2 1 6 3 4 9 5 7]
 [7 9 4 3 5 2 1 8 6]
 [3 5 8 4 6 1 7 2 9]
 [2 1 6 8 9 7 5 3 4]
 [4 8 5 2 7 6 3 9 1]
 [9 6 2 1 8 3 4 7 5]
 [1 3 7 9 4 5 8 6 2]]


In [6]:
class DataGenerator(Sequence):
    def __init__(self, df,batch_size = 16,subset = "train",shuffle = False, info={}):
        super().__init__()
        self.df = df
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.subset = subset
        self.info = info
        
        self.data_path = path
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.df)/self.batch_size))
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle==True:
            np.random.shuffle(self.indexes)
            
    def __getitem__(self,index):
        X = np.empty((self.batch_size, 9,9,1))
        y = np.empty((self.batch_size,81,1))
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        for i,f in enumerate(self.df['quizzes'].iloc[indexes]):
            self.info[index*self.batch_size+i]=f
            X[i,] = (np.array(list(map(int,list(f)))).reshape((9,9,1))/9)-0.5
        if self.subset == 'train': 
            for i,f in enumerate(self.df['solutions'].iloc[indexes]):
                self.info[index*self.batch_size+i]=f
                y[i,] = np.array(list(map(int,list(f)))).reshape((81,1)) - 1
        if self.subset == 'train': return X, y
        else: return X

In [7]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

model.add(Flatten())
model.add(Dense(81*9))
model.add(Reshape((-1, 9)))
model.add(Activation('softmax'))

adam = keras.optimizers.Adam(lr=.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 9, 9, 64)          640       
_________________________________________________________________
batch_normalization (BatchNo (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 128)         8320      
_________________________________________________________________
flatten (Flatten)            (None, 10368)             0         
_________________________________________________________________
dense (Dense)                (None, 729)               7